## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
from tqdm.notebook import tqdm

from keras.models import load_model
from keras import activations
from keras.preprocessing.image import ImageDataGenerator

from vis.visualization import visualize_saliency
from vis.utils import utils

from skimage.transform import resize
from skimage.io import imread

## Set parameters and directories
Note: This code is currently set up for generating saliency maps for all images in the test dataset

In [ ]:
# Image type "UNIL" or "CROP"
IMAGE_TYPE = "UNIL"

# File path for Excel file containing UNIL train/val/test splitting:
TRAIN_VALID_TEST_EXCEL_FILEPATH = f"............/classification_training_validation_test_{IMAGE_TYPE}.xlsx"

# File path for neural net h5 file (ensure it is the model for the correct image type UNIL or CROP as appropriate)
DIR_NN = "" 

# File path to folder containing input images (UNIL or CROP as appropriate)
INPUT_DIR = "" 

# File path to folder for saving saliency maps
OUTPUT_DIR = ""

## Load neural net and modify final dense layer to linear activation

In [ ]:
# Load neural net
saved_model = load_model(DIR_NN)

In [ ]:
# Modify final layer (dense_1) to linear activation
layer_idx = utils.find_layer_idx(saved_model, 'dense_1')
saved_model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(saved_model)

## Prepare image data generator and list of image names

In [ ]:
# Generate dataframe for the test images
images_df = pd.read_excel(TRAIN_VALID_TEST_EXCEL_FILEPATH, sheet_name=f"TEST_DF", dtype=str)

# Setup image data generator
images_datagen = ImageDataGenerator(rescale=1. / 255)
images_generator = images_datagen.flow_from_dataframe(images_df, directory=INPUT_DIR, class_mode='categorical', x_col='filenames', y_col='labels',
                                                    target_size=(299, 299), batch_size=1, shuffle=False)

In [ ]:
# Generate list of numpy arrays of images
original_images = []
originals = images_df['filenames'].to_numpy()
for original in originals:
    og_img = imread(f"{INPUT_DIR}/{original}")
    original_images.append(og_img)

## Produce saliency maps

In [ ]:
for i, (image, true) in enumerate(tqdm(images_generator)):
    og_img = np.stack((original_images[i],)*3, axis=-1)
    grads = visualize_saliency(model, 
                           layer_idx,
                           filter_indices = None,
                           seed_input = image)
    grd = np.stack((grads,)*3, axis=-1)
    gaus = resize(grd, og_img.shape, anti_aliasing = False)
    gaus = ndimage.gaussian_filter(gaus[:,:,2], sigma=(og_img.shape[0])/90)
    plt.imshow(og_img)
    plt.imshow(gaus, alpha=.6, cmap = 'viridis')
    plt.axis('off')
    plt.savefig(f'{OUTPUT_DIR}/{(originals[i])[:-4]}_SALMAP.png', dpi=300, bbox_inches="tight", pad_inches=0)
    plt.show()
    if i == 180:
        break